___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests

l=[]
date=datetime.now()

for i in range(365):
    webpage = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date={date.strftime('%Y%m%d')}&json"
    r=requests.get(webpage)
    
    if r.status_code==200:
        data=r.json()
        for currency in data:
            if currency['cc']=='USD':
                tmp=date.strftime('%Y%m'), currency["rate"]
                l.append(tmp)
           
    date-=timedelta(days=1)

2. Отримати курс долара за останній рік.

In [3]:
d = {}
for i in l:
    d.setdefault(i[0], []).append(i[1])

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [4]:
import statistics

Avg_by_month='Средний курс в разрезе месяцев: '+str({key:round(statistics.mean(values),2) for key, values in d.items()})
St_dev_by_month='Стандартное отклонение в разрезе месяцев: '+str({key:round(statistics.stdev(values),2) for key, values in d.items()})

print(Avg_by_month)
print(St_dev_by_month)

Средний курс в разрезе месяцев: {'202108': 26.77, '202107': 27.21, '202106': 27.24, '202105': 27.6, '202104': 27.93, '202103': 27.8, '202102': 27.88, '202101': 28.22, '202012': 28.17, '202011': 28.31, '202010': 28.32, '202009': 27.98, '202008': 27.44}
Стандартное отклонение в разрезе месяцев: {'202108': 0.11, '202107': 0.16, '202106': 0.16, '202105': 0.12, '202104': 0.09, '202103': 0.12, '202102': 0.11, '202101': 0.12, '202012': 0.25, '202011': 0.15, '202010': 0.06, '202009': 0.24, '202008': 0.04}


4. Дану інформацію записати у файл за допомогою pickle.

In [5]:
import pickle

try:
    file=open('курс_доллара.pickle', 'wb')
    try:
        pickle.dump([Avg_by_month,St_dev_by_month], file)
    finally:
        file.close()     
except FileNotFoundError:
    print('Невозможно открыть файл')
     
# file=open('курс_доллара.pickle', 'rb')
# check=pickle.load(file)
# file.close

# check

["Средний курс в разрезе месяцев: {'202108': 26.77, '202107': 27.21, '202106': 27.24, '202105': 27.6, '202104': 27.93, '202103': 27.8, '202102': 27.88, '202101': 28.22, '202012': 28.17, '202011': 28.31, '202010': 28.32, '202009': 27.98, '202008': 27.44}",
 "Стандартное отклонение в разрезе месяцев: {'202108': 0.11, '202107': 0.16, '202106': 0.16, '202105': 0.12, '202104': 0.09, '202103': 0.12, '202102': 0.11, '202101': 0.12, '202012': 0.25, '202011': 0.15, '202010': 0.06, '202009': 0.24, '202008': 0.04}"]

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [85]:
import requests
from bs4 import BeautifulSoup

URL='https://smallpacking.agrosem.ua/shop/'
HEADERS={
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

def get_html(url):
    r=requests.get(url, headers=HEADERS)
    return r

def get_pages_count(html):
    soup=BeautifulSoup(html, 'lxml')
    pagination=soup.find_all('a', class_='page-numbers')
    if pagination:
        return int(pagination[-2].get_text())
    else:
        return 1

def get_content(html):
    soup=BeautifulSoup(html, 'lxml')
    items=soup.find_all('div', class_='product-from-category-container')
    links=[]
    for i in items:
        links.append(i.a.get('href'))
    l=[]
    for i in links:
        soup=BeautifulSoup(get_html(i).text, 'lxml')
        l.append ((
            soup.find('span', class_='sku').text.split()[1],
            soup.find('div', class_='atributes-description').h2.text,
            soup.find('span', class_='regular-price').text.replace(" ", "").replace("грн",""),
            soup.find('span', class_='type-packaging-name').text
        ))
    return l
        
def parse():
    html=get_html(URL)
    if html.status_code==200:
        l=[]
        pages_count=get_pages_count(html.text)
        for page in range (1, pages_count+1):
            print(f'Парсинг страницы {page} из {pages_count}')
            html=get_html(URL+str('page/'+str(page)+"/"))
            l.extend(get_content(html.text))
            
            import sqlite3
            conn=sqlite3.connect('agrosem.sqlite')
            curs=conn.cursor()

            curs.execute(""" create table if not exists agrosem (
                    sku_code text not null,
                    sku_name text not null,
                    price int not null,
                    type text not null 
                ) """)

            curs.executemany(""" insert into agrosem values (?, ?, ?, ?) """, l)
            conn.commit()
    else:
        print('Error')
    
parse()

Парсинг страницы 1 из 8
Парсинг страницы 2 из 8
Парсинг страницы 3 из 8
Парсинг страницы 4 из 8
Парсинг страницы 5 из 8
Парсинг страницы 6 из 8
Парсинг страницы 7 из 8
Парсинг страницы 8 из 8


### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [13]:
import sqlite3

2. Виведіть перелік всіх таблиць.

In [50]:
conn=sqlite3.connect('tysql.sqlite')
curs=conn.cursor()

curs.execute("""
SELECT name FROM sqlite_master WHERE type='table'
""")
for result in curs:
    print(result)

('Customers',)
('OrderItems',)
('Orders',)
('Products',)
('Vendors',)


3. Список всіх cust_id з таблиці Customers table.

In [58]:
curs.execute("""
select distinct cust_id from Customers
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['cust_id']
('1000000001',)
('1000000002',)
('1000000003',)
('1000000004',)
('1000000005',)


4. Всю таблицю Customers table

In [59]:
curs.execute("""
select * from Customers
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['cust_id', 'cust_name', 'cust_address', 'cust_city', 'cust_state', 'cust_zip', 'cust_country', 'cust_contact', 'cust_email']
('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)


5. Список клієнтів (cust_names) відсортованих від Z до A

In [60]:
curs.execute("""
select cust_name from Customers
order by cust_name
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['cust_name']
('Fun4All',)
('Fun4All',)
('Kids Place',)
('The Toy Store',)
('Village Toys',)


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [61]:
curs.execute("""
select f1.cust_id, f2.order_num 
from Customers f1
left join Orders f2 on f1.cust_id=f2.cust_id
order by f1.cust_id, f2.order_date
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['cust_id', 'order_num']
('1000000001', 20009)
('1000000001', 20005)
('1000000002', None)
('1000000003', 20006)
('1000000004', 20007)
('1000000005', 20008)


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [62]:
curs.execute("""
select prod_id, quantity, item_price
from OrderItems
order by quantity desc, item_price desc
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['prod_id', 'quantity', 'item_price']
('BNBG01', 250, 2.49)
('BNBG02', 250, 2.49)
('BNBG03', 250, 2.49)
('BR03', 100, 10.99)
('BR01', 100, 5.49)
('BNBG01', 100, 2.99)
('BNBG02', 100, 2.99)
('BNBG03', 100, 2.99)
('BR03', 50, 11.49)
('RGAN01', 50, 4.49)
('BR01', 20, 5.99)
('BR03', 10, 11.99)
('BR02', 10, 8.99)
('BNBG01', 10, 3.49)
('BNBG02', 10, 3.49)
('BNBG03', 10, 3.49)
('BR03', 5, 11.99)
('RGAN01', 5, 4.99)


8. Товар (з таблиці Products), ціна якого становить 9.49

In [64]:
curs.execute("""
select prod_id, prod_name from Products where prod_price=9.49
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['prod_id', 'prod_name']
('RYL01', 'King doll')
('RYL02', 'Queen doll')


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [75]:
curs.execute("""
select prod_name, prod_price from Products where prod_price between 3.00 and 9.00 order by prod_price
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['prod_name', 'prod_price']
('Fish bean bag toy', 3.49)
('Bird bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)
('8 inch teddy bear', 5.99)
('12 inch teddy bear', 8.99)


10. Кількість товару, що було продано

In [69]:
curs.execute("""
select sum(order_item) as 'salout_qty_pcs' from OrderItems
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['salout_qty_pcs']
(45,)


11. Кількість найменувань товару, ціна якого більша за 4

In [72]:
curs.execute("""
select count(distinct prod_name) as 'count_products' from Products where prod_price > 4
""") ##count(prod_id) но я так понял, что "найменувань" -  это именно prod_name
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['count_products']
(6,)


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [79]:
curs.execute("""
select prod_id, prod_name from Products order by prod_price desc limit 3
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['prod_id', 'prod_name']
('BR03', '18 inch teddy bear')
('RYL01', 'King doll')
('RYL02', 'Queen doll')


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [81]:
curs.execute("""
select count_order_num, cust_id from (
    select count(order_num) as count_order_num, cust_id from Orders group by cust_id
) f1
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['count_order_num', 'cust_id']
(2, '1000000001')
(1, '1000000003')
(1, '1000000004')
(1, '1000000005')


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [82]:
curs.execute("""
select f1.cust_id, f1.cust_name, f2.order_num
from Customers f1 inner join Orders f2 on f1.cust_id=f2.cust_id
""")
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['cust_id', 'cust_name', 'order_num']
('1000000001', 'Village Toys', 20005)
('1000000003', 'Fun4All', 20006)
('1000000004', 'Fun4All', 20007)
('1000000005', 'The Toy Store', 20008)
('1000000001', 'Village Toys', 20009)


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [89]:
curs.execute("""
select distinct prod_id, prod_name from Products where prod_price < 5
union
select distinct prod_id, prod_name from Products where prod_price >= 5
""") ##count(prod_id) но  я так понял, что "найменувань" -  это именно prod_name
name = list(map(lambda x: x[0], curs.description))
print(name)
for result in curs:
    print(result)

['prod_id', 'prod_name']
('BNBG01', 'Fish bean bag toy')
('BNBG02', 'Bird bean bag toy')
('BNBG03', 'Rabbit bean bag toy')
('BR01', '8 inch teddy bear')
('BR02', '12 inch teddy bear')
('BR03', '18 inch teddy bear')
('RGAN01', 'Raggedy Ann')
('RYL01', 'King doll')
('RYL02', 'Queen doll')


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


#### Задание: 1 (Serge I: 2002-09-30)
Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd

select prod.model, speed, hd  
from Product prod  
inner join PC pc on prod.model=pc.model  
where pc.price<500

#### Задание: 2 (Serge I: 2002-09-21)
Найдите производителей принтеров. Вывести: maker

select distinct maker  
from Product  
where type ='Printer'

#### Задание: 3 (Serge I: 2002-09-30)
Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.

select model, ram, screen  
from Laptop  
where price>1000

#### Задание: 4 (Serge I: 2002-09-21)
Найдите все записи таблицы Printer для цветных принтеров.

select *  
from Printer  
where color='y'

#### Задание: 5 (Serge I: 2002-09-30)
Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.

select model, speed, hd  
from PC  
where price<600  
and cd in ('12x', '24x')  

#### Задание: 6 (Serge I: 2002-10-28)
Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость.

select distinct f1.maker,f2.speed  
FROM Product f1  
inner join Laptop f2 on f1.model = f2.model   
where f2.hd >= 10  

#### Задание: 7 (Serge I: 2002-11-02)
Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква).

select distinct product.model, pc.price  
from Product inner join pc on product.model = pc.model where maker = 'B'  
union  
select distinct product.model, laptop.price  
from product inner join laptop on product.model=laptop.model where maker='B'  
union  
select distinct product.model, printer.price  
from product inner join printer on product.model=printer.model where maker='B'  

#### Задание: 8 (Serge I: 2003-02-03)
Найдите производителя, выпускающего ПК, но не ПК-блокноты.

select distinct maker  
from Product f1  
where (  
  select count (1)   
  from product f2  
  where f2.type='PC'   
  and f2.maker=f1.maker  
) > 0 AND (  
  select count (1)   
  from product f2  
  where f2.type = 'Laptop'  
  and f2.maker = f1.maker  
) = 0  

#### Задание: 9 (Serge I: 2002-11-02)
Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker

select distinct maker  
from Product f1  
inner join PC f2 on f1.model=f2.model  
where speed>=450  

#### Задание: 10 (Serge I: 2002-09-23)
Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price

select model, price  
from Printer  
where price in (select max(price) from Printer)  

#### Задание: 11 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК.

select avg(speed)
from PC

#### Задание: 12 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол.

select avg(speed)  
from Laptop  
where price>1000  

#### Задание: 13 (Serge I: 2002-11-02)
Найдите среднюю скорость ПК, выпущенных производителем A.

select avg(f1.speed)  
from PC f1   
inner join Product f2 on f1.model=f2.model   
where f2.maker='A'   

#### Задание: 14 (Serge I: 2002-11-05)
Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий.

select f1.class, f1.name, f2.country  
from Ships f1  
inner join Classes f2 on f1.class=f2.class  
where f2.numGuns>=10  

#### Задание: 15 (Serge I: 2003-02-03)
Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD

select hd  
from PC  
group by hd  
having count(model)>=2  

#### Задание: 16 (Serge I: 2003-02-03)
Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.

select distinct f1.model, f2.model, f1.speed, f1.ram  
from pc f1, pc f2     
where f1.speed=f2.speed   
and f1.ram=f2.ram   
and f1.model>f2.model  

#### Задание: 17 (Serge I: 2003-02-03)
Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК.
Вывести: type, model, speed

select distinct f2.type, f1.model, f1.speed  
from Laptop f1  
inner join Product f2 on f1.model=f2.model  
where f1.speed<(select min(speed) from PC)  

#### Задание: 18 (Serge I: 2003-02-03)
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price

select distinct product.maker, printer.price  
from product, printer  
where product.model = printer.model  
and printer.color = 'y'  
and printer.price = (  
select min(price) from printer  
where printer.color = 'y'  
)  

#### Задание: 19 (Serge I: 2003-02-13)
Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов.
Вывести: maker, средний размер экрана.


select product.maker, avg(screen)  
from laptop  
left join product on product.model = laptop.model  
group by product.maker  

#### Задание: 20 (Serge I: 2003-02-13)
Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

select maker, count(model)  
from product  
where type ='pc'  
group by product.maker  
having count (distinct model)>=3  

#### Задание: 21 (Serge I: 2003-02-13)
Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC.
Вывести: maker, максимальная цена.

select product.maker, max(pc.price)  
from product, pc   
where product.model = pc.model  
group by product.maker  

#### Задание: 22 (Serge I: 2003-02-13)
Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена.

select pc.speed, avg(pc.price)  
from pc   
where pc.speed > 600  
group by pc.speed  

#### Задание: 23 (Serge I: 2003-02-14)
Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker

select distinct maker  
from product t1 inner join pc t2 on t1.model=t2.model  
where speed>=750 and maker in(  
select maker  
from product t1 inner join laptop t2 on t1.model=t2.model  
where speed>=750  
)  

#### Задание: 24 (Serge I: 2003-02-03)
Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

select model  
from (  
 select model, price  
 from pc  
 union  
 select model, price  
 from Laptop  
 union  
 select model, price  
 from Printer  
) t1  
where price = (  
 select max(price)  
 from (  
  select price  
  from pc  
  union  
  select price  
  from Laptop  
  union  
  select price  
  from Printer  
  ) t2  
 )  

#### Задание: 25 (Serge I: 2003-02-14)
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

select distinct maker  
from product  
where model in (  
select model  
from pc    
where ram = (  
  select min(ram)  
  from pc  
  )   
and speed = (  
  select max(speed)  
  from pc  
  where ram = (  
   select MIN(ram)  
   from pc  
   )  
  )  
)  
and  
maker in (  
select maker  
from product  
where type='printer'  
)  

#### Задание: 26 (Serge I: 2003-02-14)
Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

select sum(s.price)/sum(s.kol) as sredn from  
(select price,1 as kol from pc,product  
 where pc.model=product.model and product.maker='A'  
union all  
 select price,1 as kol from laptop,product  
 where laptop.model=product.model and product.maker='A') as s  

#### Задание: 27 (Serge I: 2003-02-03)
Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD.

select product.maker, avg(pc.hd)   
from pc, product where product.model = pc.model  
and product.maker in (select distinct maker  
from product  
where product.type = 'printer')  
group by maker  

#### Задание: 28 (Serge I: 2012-05-04)
Используя таблицу Product, определить количество производителей, выпускающих по одной модели.

select count(maker)  
from (  
select maker, count(distinct model) wtf  
from Product  
group by maker  
having count(distinct model)=1  
) f1  

#### Задание: 29 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o.

select t1.point, t1.date, inc, out  
from income_o t1 left join outcome_o t2 ON t1.point = t2.point  
and t1.date = t2.date  
union  
select t2.point, t2.date, inc, out  
from income_o t1 right join outcome_o t2 ON t1.point = t2.point  
and t1.date = t2.date  

#### Задание: 30 (Serge I: 2003-02-14)
В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).

select point, date, sum(sum_out), sum(sum_inc)  
from( select point, date, sum(inc) as sum_inc, null as sum_out from Income group by point, date  
union  
select point, date, null as sum_inc, sum(out) as sum_out from Outcome group by point, date ) as t  
group by point, date order by point  